# **Chap9：对抗和生成**

## **9.1 对抗攻击**

深度学习模型凭借成功的架构设计、互联网技术发展带来的大量可用的训练数据、以及高速发展的专用芯片计算技术在各领域取得了巨大成功，各种模型正在被部署到实际的社会生产生活中，例如图像识别、语音识别、生物基因等，**当人们的生产、生活越来越多地依赖这些人工智能模型时，模型的安全性就成为了应用人工智能模型必须考虑的部分**

在机器学习中，研究模型安全性的领域通常被归类为**对抗攻击和防御**（**Adversarial Attacks & Defences**），攻防在任何安全领域都将作为两个重要方面进行讨论，在对抗攻击方面，人们已经找到攻击机器学习模型的各种方法，使其失去有效性，甚至可以诱导模型产生攻击者想要获得的结果

本节我们主要介绍对抗攻击的各种基础概念，初步领略“对抗”的概念和思想，这便于我们理解后续介绍的对抗学习算法

### **9.1.1 对抗威胁分类**

要清楚对抗攻击的各种手段，我们首先可以抽象一个机器学习模型部署运作的流程，任何基于机器学习模型的智能系统可以被看作一套数据处理的流程，在**测试阶段**，整个处理过程按顺序可以被分为：
- (a) 从传感器或数据仓库收集输入数据 `input data`
- (b) 将收集到的数据转换为数字信息存储，例如一张 `.jpg` 的图像
- (c) 预处理数字信息，使其变换为模型所需要的张量
- (d) 应用机器学习模型，得到模型输出结果，例如分类概率
- (e) 根据模型输出的结果采取相应的行动

以交通警示标志的识别为例，上述过程的示意图如下所示：

<img src="../source/Chap9/交通警示标志识别流程.jpg" width=900 style="display: block; margin-left: auto; margin-right: auto;">

在上述流程中，交通标志的图像被传感器或照相机采集得到 `.jpeg` 数字图像，然后经过数据预处理将图像转换为模型所需要的像素张量，模型基于该输入得到有关标识识别的概率分布，最后根据概率分布采取相应的行为（例如停车）

我们将整个模型运作流程定义为**对抗接触面**（**Attack Surface**），任何攻击者将会尝试在数据采集、或数据处理的环节中进行干扰，最终导致目标模型产生错误的结果，根据对抗接触面，攻击类型可以大致被分为三类：
- **(1) 回避攻击**（**Evasion Attack**）：这是在对抗领域最常见的一种攻击，攻击者将试图在模型预测阶段，通过调整恶意样本来规避机器学习系统的检测，回避攻击不会影响模型的训练，也不会影响训练数据
- **(2) 毒害攻击**（**Poisoning Attack**）：该类型的攻击发生在模型的阶段，攻击者试图扰动模型训练数据集，或者扰动模型本身（给数据、模型“下毒”），这可以通过在模型学习过程中注入精心设计的有害样本来实现
- **(3) 探索性攻击**（**Exploratory Attack**）：这类攻击同样不影响训练数据集，通过给定对模型的**黑盒访问**，攻击者试图从模型算法系统背后包含的尽可能多的信息，这些信息可能会泄漏训练数据集的模式

#### **(A) 攻击能力分类**

我们可以根据攻击者所掌握有关机器学习系统的信息，以及攻击者可触及到系统的渗透程度对攻击者的能力进行划分
* 例如一个**内部攻击者**可以控制模型的训练，访问到模型的架构，而一个**薄弱攻击者**只能够在模型测试阶段，向模型提供一些样本以获得相应的预测结果
* 两种攻击者面对的是相同的攻击接触面（即相同的模型应用工作流），但内部攻击者显然拥有模型更丰富的信息，因此也被视为“更强的攻击者”

我们可以先**从模型的训练 / 预测阶段对攻击能力进行分类**

**(1) 训练阶段攻击能力**

在训练阶段出手的攻击者，将尝试通过改变训练数据集直接影响或干扰模型，训练阶段最直接、最弱的攻击是**只访问部分或全部训练数据**，通过攻击者的能力，对模型的攻击扰动可以被分为下面三类：
* **(a) 数据注入**：攻击者**不能访问到训练数据集和模型结构，学习算法**，但是其**有能力在数据集中添加新的样本**，攻击者可以通过在数据集中**注入对抗样本**来扰乱目标模型
* **(b) 数据篡改**：攻击者**不能访问模型结构和学习算法**，但是可以**完整访问模型训练数据集**，攻击者可以通过在讲数据提供给模型之前，直接修改数据样本以实施**毒害攻击**
* **(c) 逻辑破坏**：攻击者**有能力干预学习算法**，显然，针对这些可以改变学习逻辑从而控制模型本身的攻击者，设计对抗防守策略变得非常困难

**(2) 测试阶段攻击能力**

**测试阶段的对抗攻击者不会尝试篡改目标模型**，而是尝试迫使其产生不正确的预测结果，这类攻击的强度主要依赖于攻击者对目标模型掌握的信息量，测试阶段的攻击可以被简单地分类为**白盒攻击**（**White-Box Attack**）和**黑盒攻击**（**Black-Box Attack**）

为了方便后续讨论，我们先对建模过程做一些形式化说明，假设目标机器学习模型为 $f$，它在输入样本对 $(X,y)$ 上训练得到，而 $(X,y)$ 抽样自训练数据分布 $\mu$，整个训练过程是一个随机化的训练算法 $\text{train}$，随机性记为 $r$ （例如神经网络随机初始化的权重，dropout 等带来的随机性），模型的参数 $\theta$ 可以写为：

$$
\theta \leftarrow \text{train}\left(f,X,y,r\right)
$$

* **白盒攻击**

白盒攻击设定下，攻击者拥有关于模型 $f$ 的完整知识（例如神经网络的结构）、关于训练算法 $\text{train}$ 的信息（例如知道使用了 SGD 优化器），并且可以访问训练数据分布 $\mu$，攻击者还可以获取到训练模型的参数 $\theta$

攻击者可以利用这些信息来**识别模型可能易受攻击的特征空间区域**（即模型具有高错误率的特征空间），然后通过使用**对抗样本制作算法**更改输入来影响模型预测，白盒攻击对内部模型权重 $\theta$ 的访问是一种非常强的对抗攻击，后文将介绍一些对抗样本制作算法的概念

* **黑盒攻击**

相反，黑盒攻击假设攻击者不了解模型结构，没有用于训练模型的信息，也不知道训练模型的历史输入，**整个模型对于攻击者是一个黑箱**，攻击者只能观察给定输入下模型的输出结果，黑盒攻击可以分为以下几大类：
- **(1) 非自适应黑盒攻击**：这类攻击假设攻击者可以获得目标模型训练数据的分布 $\mu$，攻击者将选择一种训练学习算法 $\text{train}'$ 和一种模型结构 $f'$，然后从分布 $\mu$ 中获取样本训练一个本地模型 $f'$ 去逼近目标模型 $f$，然后攻击者通过 $f'$ 利用白盒攻击策略来制作对抗样本，并将制作的样本用于目标模型 $f$ 以迫使其做出错误预测
- **(2) 自适应黑盒攻击**：对于目标模型 $f$，攻击者不能获得有关训练过程的任何信息，但是可以方便地访问模型 $f$，这种攻击策略类似于密码学中的**选择明文攻击**，攻击者向 $f$ 发出自适应查询并标记精心选择的数据集（即对于任意选择的 $x$，通过查询目标模型 $f$ 获得其标签 $y$），然后，攻击者选择一个训练算法和模型结构 $f′$，在查询得到的样本 $(x,y)$ 上训练代理模型 $f'$，随后在代理模型 $f'$ 上使用各种白盒攻击策略，就可以生成对抗样本欺骗目标模型 $f$
- **(3) 严格黑盒攻击**：在该设定下，攻击者同样无法获得训练数据集分布 $\mu$，同样只能从目标模型中收集样本对 $(x,y)$，但**与自适应黑盒攻击的区别在于，攻击者无法任何修改提供给目标模型的输入来观察输出的改变**，这种策略类似于密码学中的**已知明文攻击**，这通常需要大量的输入输出样本对才能获得成功

对于黑盒攻击者而言，要**明确攻击者不会尝试学习随机性** $r$ **以及目标模型的参数** $\theta$，黑盒攻击者的主线策略是收集尽可能多的信息，然后训练一个本地代理模型（非自适应下使用分布 $\mu$，自适应下通过向目标模型发起精心设计的查询构造样本对）

下表简单总结了黑盒攻击和白盒攻击的区别

<img src="../source/Chap9/白盒黑盒攻击对比.png" width=700 style="display: block; margin-left: auto; margin-right: auto;">

#### **(B) 攻击目的分类**

除了按照攻击能力分类，我们还可以按照攻击者的攻击意图和会采取的行动分类，这将决定攻击者会多大程度上影响模型，根据模型输出的被影响程度和攻击目标，对抗攻击可以划分为以下四类：
- **(1) 置信度下降**：攻击者试图**降低目标模型输出预测概率分布的置信度**，例如，停车标识经过攻击后可能会极大地降低其置信度，使得目标模型输出概率分布中属于停车标识的一类概率值降低
- **(2) 错误分类**：攻击者对于给定的输入样本，尝试更改模型的分类结果，**使其分类结果异于原始的正确答案**，例如，停车标识经过攻击后会被错误地分类为任何除停车标识以外的其它标识
- **(3) 指定输出的错误分类**：攻击者不但要使模型输出错误的分类结果，还要**迫使其输出指定的错误结果**，例如，攻击者会攻击目标模型使其**将任何标识输入预测为通行标识**
- **(4) 指定输入输出的错误分类**：攻击者试图**使模型针对特定的输入，输出指定的错误预测**，例如，攻击者会攻击目标模型使其将一个停车标识错误预测为通行标识

下图展示了分别对于**规避攻击**（**Evasion Attacks**）和**毒害攻击**（**Poisoning Attacks**），攻击的困难程度随着攻击者能力和攻击意图的变化趋势

<img src="../source/Chap9/规避攻击和毒害攻击的难度.png" width=900 style="display: block; margin-left: auto; margin-right: auto;">

在上图中，横轴表示攻击者的能力和强度，纵轴代表攻击目的复杂程度，对角线方向表示了基于不同的攻击能力和想达到的攻击目标，所对应攻击任务的困难度

----

### **9.1.2 探索性攻击**

探索性攻击不会更改训练数据集，而是试图获得有关提供给学习器和模型的相关信息，攻击者试图创建对抗样本，使得目标模型会将其视为合法样本进行传递，下面介绍三种常见的探索性攻击策略

* **模型反演攻击**（**Model Inversion**）

Fredrikson 等人提出了模型反演攻击策略，他们考虑了一个使用病患信息，药物史和基因标记来预测药物剂量的线性回归模型 $f$，当给定模型的白盒访问以及数据实例 $(X,y)$ 时，模型反演将尝试推断病患的基因标记 $x_1$. 该算法通过迭代特征 $x_1$ 的所有可能取值来获得目标值 $y$，从而产生 $x_1$ 的**最小偏差-最大先验（MAP）估计**，从而**最小化攻击误预测率**

这种简单的策略有很大局限性，当未知特征集合非常大时，穷尽所有可能在计算复杂度上将变得不可行. Fredrikson 尝试改进并移除之前工作的局限性，并展示了对于黑盒攻击者同样可以预测病患的基因标记. 这种新的模型反演策略**通过 ML API 进行攻击**，这些 API 在各种设置中利用置信值. 这类攻击可以推断出生活方式调查决策树模型的敏感特征输入，或者从 API 访问面部识别服务中恢复输入图像

在基于神经网络的面部识别任务中已经有很多成功攻击的策略模型，例如：**Softmax 回归，多层感知机 MLP，堆叠的去燥自编码网络 DAE（Denoising AutoEncoder）**，给定模型的访问访问并提供某个人的名字，攻击者可以恢复出人脸图像，如下图所示

<img src="../source/Chap9/面部识别系统的攻击.jpg" width=800 style="display: block; margin-left: auto; margin-right: auto;">

但由于深度学习的丰富结构，模型反演攻击**可能恢复出与定义类的实际数据几乎不相似的原型示例**（**Prototypical**）

* **利用 API 的模型提取**

Tramer 等人提出了提取热门机器学习模型类（如逻辑回归、神经网络和决策树）的简单攻击策略，所提出的攻击是**严格的黑匣子攻击**，但可以**在功能上接近在本地构建目标模型**. Tramer 演示了对 BigML 和 Amazon 机器学习等在线 ML 服务提供商的模型提取攻击. ML-as-service 提供商提供的机器学习 API 返回置信值以及类标签，由于攻击者没有任何关于模型或训练数据分布的信息，因此攻击者只能尝试通过查询未知 $d+1$ 个模型参数的 $d+1$ 个随机 $d$ 维输入获得模型输出的置信值，来**对未知参数或特征相关的方程进行数学求解**

* **推断攻击**（**Inference Attack**）

Ateniese 等人提出，从机器学习分类器中收集相关隐私相关的信息是可能的. **给定对模型的黑盒访问**（例如，通过公共 API）和**一个训练样本**，攻击者可能想知道**该数据是否是模型训练数据集的一部分**，Ateniese 试验了一种使用隐马尔可夫模型的语音识别分类器，并提取了用户的口音等信息，而这些隐私信息本不应该被明确捕获

Shokri 等人提出的另一种推断攻击是**隶属推理**（**Membership Inference**），攻击者试图**确定给定数据点是否与训练数据集属于相同的分布**. 这种攻击可能属于非自适应或自适应黑盒攻击，在典型的黑盒环境中，攻击者向目标模型发送带有数据点的查询，并获得模型的预测结果. 模型给出的输出是一个概率向量，用于指定数据点是否属于某一类，为了训练攻击模型，攻击者需要建立影子模型去模拟目标模型的行为，对于影子模型而言，攻击者知道给定记录是否属于训练集，因此可以采用监督学习，将相应的输出标签提供给攻击模型进行训练，以区分影子模型对其训练数据成员的输出与非成员的输出

一套端对端的隶属推理的流程图如下图所示，训练逻辑为：
- (1) 对于影子模型训练数据集中的样本，将影子模型对它们的输出概率向量记为 $x$，**标签 $y$ 设定为 "in"**，将样本对 $(x,y)$ 添加到攻击模型的训练数据集
- (2) 一个样本外的测试数据集同样提供给影子模型，此时影子模型的输出概率向量记为 $x$，**标签 $y$ 设定为 "out"**，将样本对 $(x,y)$ 添加到攻击模型的训练数据集
- (3) 致辞，攻击模型**通过二分类任务来区分影子模型的黑盒行为**

<img src="../source/Chap9/隶属推断.jpg" width=800 style="display: block; margin-left: auto; margin-right: auto;">

这种基于分类任务的隶属推理攻击模型可以在通常的 ML-as-service 提供商（例如 Google 和 Amazon）服务上训练得到

----

### **9.1.3 规避攻击和毒害攻击**

规避攻击是机器学习系统中最常见的攻击类型，恶意样本被精心设计，从而迫使模型做出错误预测，规避检测. 而毒害攻击区别在于，对输入的篡改可以在训练阶段进行，从而模型在被污染的数据集上训练得到，使模型在测试阶段获得攻击者想要的输出结果

#### **9.1.3.1 生成对抗攻击**（**Generative Adversarial Attack，GAN**）

Goodfellow 等人提出了**生成对抗神经网络**（**Generative Adversarial Network，GAN**），用于生产与训练数据集相似的样本，以获得与训练数据集几乎相同的分布. GAN 模型由一个判别器深度网络 $D$ 和生成器深度网络 $G$ 组成，判别期的工作在于区分从原始数据集中采样的样本和生成器 $G$ 生成的虚假样本（即 $D$ 是一个二分类器），生成器 $G$ 以随机噪声作为输入，然后生成尽可能与训练数据集分布一致的样本欺骗判别器 $D$ 使其无法区分

从公式上来看，我们需要训练 $G$ 以最大化 $D$ 犯错误的概率，同时训练 $D$ 以最小化二分类的交叉熵损失，**这场 $D$ 与 $G$ 的竞赛引导两个模型不断提升它们各自的性能，整个算法流程在 $D$ 失去区分能力时停止**，此时意味着生成器 $G$ 能够生成与训练数据集分布完全相同的样本，GAN 的目标函数 $V(G,D)$ 如下：

$$
\mathop{\min_G}\mathop{\max_D} V(D,G) = \mathbb{E}_{x\sim p_{\text{data}}(x)} \log D(x) + \mathbb{E}_{z\sim p_z}\log (1 - D(G(z)))
$$

其中，$p_{\text{data}}$ 是真实的数据分布，$p_z$ 是一个生成器输入噪声的先验分布，例如多维正态分布，下图展示了生成对抗学习的示意图，**有关 GAN 更详细的介绍和讨论在 9.3 节中进行**

<img src="../source/Chap9/生成对抗学习示意图.jpg" width=800 style="display: block; margin-left: auto; margin-right: auto;">

#### **9.1.3.2 对抗性样本生成**

对抗性样本生成通过篡改样本以使得模型做出错误预测，这种对样本的篡改可以发生在训练阶段和测试阶段，下面做简单的介绍

**(A) 训练阶段篡改**

模型的学习过程通过分析训练集来微调假设的参数 $\theta$，这使得训练集容易受到攻击者的操纵. Barreno 等人继 Kearns 等人的工作之后，首次提出了“毒害攻击”，毒害攻击通过插入、修改或删除样本点来改变训练数据集，以达到修改目标模型决策边界的目的，从而挑战机器学习系统的完整性和安全性. 训练集的毒害攻击可以通过两种方式进行：**直接修改训练数据的标签**，或者**根据攻击者的能力篡改输入特征**
* **操纵标签**：如果攻击者只能够操纵训练样本的标签，在给定全部或者部分模型的知识时，攻击者必须找到最脆弱的样本标签（例如 SVM 决策超平面附近的标签，因为它们构成了支持向量）. 一种最基本的策略是随机扰乱训练标签，即通过从随机分布中抽样为训练样本或样本的一部分选择新的标签
* **操纵输入**：在这种场景下，攻击者能力更强，可以直接操纵修改输入样本的特征. 在该场景下，通常假设攻击者还能够获得学习算法的相关信息. Kloft 等人展示了通过在训练数据集中插入恶意样本，能够不断移动异常检测分类器的决策边界. 他们假设了一种**在线学习场景**，模型在有规律的间隔时间段收集新的样本，然后模型参数在新样本上进行微调, 这样，在训练数据集中注入有害样本对于攻击者而言将变得非常容易. 而在**离线训练场景下**，Biggio 等人提出了一种，利用梯度上升方法构造恶意样本的样本注入攻击策略. 该策略尝试找到输入关于模型测试误差的局部极大值. Mei 等人介绍了一个通用的毒改攻击框架，只要使用凸优化损失训练目标模型并且输入空间是连续空间时，攻击者就可以发现训练集的最佳变化以构造恶意样本

**(B) 测试阶段篡改**

我们分白盒攻击和黑盒攻击来讨论，主要的攻击策略都是基于白盒假设，而黑盒攻击的核心在于找到一个模型的白盒代理，然后再使用白盒攻击策略

* **白盒攻击**：对于白盒场景，构造对抗样本的通用的攻击策略可以拆分为两个步骤：**方向敏感性估计**和**扰动选择**，如下图所示
    <img src="../source/Chap9/白盒攻击对抗样本生成.jpg" width=900 style="display: block; margin-left: auto; margin-right: auto;">

    图中展示了基于神经网络的图像分类任务对抗样本的制作过程，该流程可以被推广到任何有监督学习任务中. 假设 $X$ 是输入样本，$F$ 是训练得到的分类器模型，攻击者的目标是通过在原始样本 $X$ 的基础上增加扰动 $\delta X$ 制作一个恶意样本 $X_* = X + \delta X$，从而 $F(x_*)=Y_*$ 而 $Y_*\ne F(X)$. 因为在白盒场景下，攻击者可以访问到训练后模型 $F$ 的参数 $\theta$，攻击者可以通过一个两步过程来制作对抗样本
    * **方向敏感性估计**：攻击者通过评估样本 $X$ 周围的数据流形，来评估类别变化对每个输入特征的敏感性. 然后在样本 $X$ 中，确认哪些部分对于改变模型 $F$ 预测类别是最敏感的
    * **扰动选择**：随后，攻击者利用**方向敏感性估计**得到的知识和信息，在所有输入维度上选择扰动 $\delta X$，以达到最高效的扰动目标

    以上两步在新一轮迭代中，通过替换 $X = X+\delta X$ 交替进行，直到攻击目标满足. 
    
    需要记住，在该设定下，对原始样本 $X$ 的总扰动量应该尽可能减小，例如在图像识别领域，对抗样本应该保持在人眼看来没有区别，而如果通过大量的扰动以取得攻击目标则失去意义. 
    
    因此，我们可以定义一个范数 $\|\cdot \|$ 来度量原始样本和对抗样本之间的差异，则寻找优秀对抗样本可以被形式化为下面的最优化问题：
    $$
    X_* = X + \mathop{\arg\min}\limits_{\delta X} \left\{ \|\delta X\| : F(X + \delta X) \ne F(X) \right\}
    $$

    大多数神经网络模型使上述公式具有**非线性和非凸性**，使得在大多数情况下很难找到解析解，下面简单介绍上述两个步骤的常见策略：
    * **方向敏感性估计**：在该步骤，攻击者的目标是找到能够帮助产生逾期对抗表现的 $X$ 的特征方向，并尽可能以最小的扰动实现. 这可以通过改变 $X$ 的输入成分，然后评估模型 $F$ 输出关于这些改变的敏感性得到. 
        * **L-BFGS**：Szegedy 等人以下式的最小化问题第一次提出对抗样本的概念，对抗样本的搜索规则为：
            $$
            \mathop{\arg\min}\limits_{r} f(x+r) = l \quad \text{s.t.} \quad (x+r)\in D
            $$

            输入样本 $x$（能够被模型 $f$ 正确分类为 $h(x)$）被施加扰动 $r$ 后得到对抗样本 $x_* = x + r$，攻击者需要保证扰动后样本 $x_*$ 依然在合法的输入空间 $D$ 中，但是此时模型预测错误 $f(x_*) = l\ne h(x)$. 对于神经网络一类的非凸模型，作者使用 L-BFGS 优化算法求解上式. 虽然这种方法具有很好的表现，但是对抗样本的计算在计算上非常昂贵（L-BFGS 需要二阶梯度）
        * **快速梯度标记**（**Fast Gradient Sign Method，FGSM**）：Goodfellow 等人提出了一种求解上述优化问题的高效方法，通过计算神经网络损失函数关于输入的梯度，对抗样本可以用下式生成：
            $$
            X_* = X + \epsilon\cdot \text{sign}\left( \nabla_x J(X,y_{\text{true}}) \right)        
            $$

            其中，$J$ 目标模型的损失函数，$\nabla_x$ 是模型关于输入样本 $X$ 和正确标签 $y_{\text{true}}$ 的梯度算子，$\epsilon$ 是输入扰动的强度. FGSM 被提出后很快有一些变种，例如：
            * **Target Class Method**：如果攻击者希望对抗样本的预测结果为指定类别 $y_{\text{target}}$，则对抗样本的生成可以更改为：
                $$
                X_* = X - \epsilon\cdot \text{sign}\left( \nabla_x J(X,y_{\text{target}}) \right)        
                $$
            * **Basic Iterative Method**：该方法是对 FGSM 的简单扩展，通过一个步长参数迭代生成对抗样本：
                $$
                X_*^0 = X;\quad X_*^{n+1} = \text{Clip}_{X,e}\left\{X_*^n + \alpha \cdot \text{sign}\left( \nabla_x J(X_*^n, y_{\text{true}}) \right) \right\}        
                $$

                $\alpha$ 是步长参数，$\text{Clip}_{X,e}{A}$ 表示逐元素的截断 $X$，元素 $A_{i,j}$ 被截断后需要在区间 $\left[X_{i,j}-e, X_{i,j}+e\right]$ 中，这种方法通常不依赖于模型的任何近似，并且在进行多轮迭代时会产生额外的对抗样本
        * **基于 Jacobian 矩阵的方法**：Papernot 等人提出了另一种使用目标模型 $F$ 的 Jacobian 矩阵的前向梯度的方法. 该方法直接获得输出成分对应于每个输入成分的梯度，这些信息通过一个复杂的**显著性图**帮助攻击者构造对抗样本. **这类方法对于指定输入输出的错误分类攻击尤其有效**
    * **扰动选择**：攻击者现在可以使用关于输入差异的灵敏度信息，以评估最有可能以最小扰动产生目标误分类的维度，下面是两种常用的扰动方案：
        * **全维度扰动**：基于 FGSM 方法，Goodfellow 等人提出的扰动在所有输入维度上进行，不过仅需在计算得到的梯度符号方向上做一个较小的扰动，这类方法高效地最小化原始样本和对抗样本之间的欧几里得距离
        * **选择部分输入维度扰动**：Papernot 等人的方法依赖于一个**显著性图**（**Saliency Maps**），通过一个较复杂的选择过程构造对抗样本，但这种方法只需要选择有限数量的输入维度进行扰动. 使用显著性图的目的是为输入维度的组合分配值，该值指示如果组合受到干扰，是否会有助于攻击目标. 
        
        这种方法有效地减少了构造对抗样本所需对原始输入进行扰动的维度数. 为了选择扰动的输入维度，所有维度都按照对抗显著性值的递减顺序进行排序，对于给定样本 $x$ 和目标类别 $t$，某个分量 $i$ 显著性值 $X(x,t)[i]$ 的计算方式为：
            $$
            S(x,t)[i] = \left\{
                \begin{array}{ll}
                0, & \displaystyle{\text{if}\quad \frac{\partial F_t}{\partial x_i}(x) < 0\quad\text{or}\quad \sum_{j\ne t}\frac{\partial F_j}{\partial x_i}(x) > 0 } \\
                \displaystyle{ \frac{\partial F_t}{\partial x_i} (x)\big|\sum_{j\ne t} \frac{\partial F_j}{\partial x_i} (x) \big|}, & \text{otherwise}
                \end{array}
            \right.
            $$
            其中，$\left[\partial F_j / \partial x_i\right]_{i,j}$ 可以利用目标模型 $F$ 的 Jacobian 矩阵 $J_F$ 快速得到，然后输入特征按照显著性值 $S(x,t)[i]$ 的递减顺序添加扰动 $\delta x$，直到 $x_* = x+\delta x$ 能够呗模型 $F$ 错误分类
        
        两种方法各有千秋，第一种方法可以简单快速地构造对抗样本，但是由于对所有输入维度都进行了扰动，因此可能比较容易被检测出来；第二种方法仅对少量输入维度施加变换，但需要付出高昂的计算代价

* **黑盒攻击**：对于非自适应黑盒攻击者，严格黑盒攻击者，攻击的策略非常直接，攻击者可以访问充足的数据集然后在本地训练一个代理模型去逼近目标模型的决策边界，**一旦本地模型经过训练后拥有较高的置信度，任何白盒攻击策略就可以在本地的代理模型上应用从而生成对抗样本**，因为**神经网络的可迁移性质**，这些对抗样本通常可以欺骗目标模型. 而对于自适应黑盒攻击场景，攻击者无法访问到大量数据集，因此只能通过数据增强的手段，通过向目标模型发起有针对性的查询，尽可能获得足够多信息
    * **基于 Jacabian 的数据增强**：一个攻击者可以不断向目标模型发起查询以获得无限多的样本对，但虽然能够帮助攻击者获得一份模型的复制，但发起大量查询的行为本身就很异常，容易被检测到. 
    
        一种策略是，基于识别目标模型在厨师训练样本点周围的输出变化方向，启发式地构建更多的训练输入，当有了更多的输入-输出样本对后，目标模型的变化方向可以被更容易捕获. 因此，攻击者的贪心启发式策略是在向目标模型发起查询时，对样本做优先级排序，在本地训练获得替代模型 $F$ 以逼近目标模型的决策边界，
        
        模型的变化方向可以利用替代模型 $F$ 的 Jacobian 矩阵 $J_F$ 得到（在一些输入样本点 $x$ 上计算）. 更详细地，攻击者可以评估 Jacobian 矩阵维度的符号 $\text{sign}\left(J_F(x)[O(x)]\right)$，$O(x)$ 是查询目标模型得到的输出，作为标签. 随后，攻击者可以在原样本的基础上添加更新方向，以获得新的训练样本点，该数据增强过程可以迭代循环，以获得更多的训练样本：
        $$
        S_{n+1} = \big\{ x + \lambda\cdot \text{sign}\left(J_F(x)[O(x)]\right) : x\in S_n\big\} \cup S_n
        $$

        其中，$S_n$ 是第 $n$ 步的数据集，$S_{n+1}$ 是数据扩充后的新数据集，整个替代模型的训练过程如下图所示：

        <img src="../source/Chap9/Jacobian数据扩充.jpg" width=900 style="display: block; margin-left: auto; margin-right: auto;">
    
    * **对抗样本的可迁移性**：可迁移性指基于某个模型创建的对抗样本可能对其他模型也具有攻击效果，这是黑盒攻击者成功的基础. 在黑盒攻击场景，攻击者无法访问目标模型 $F$ 的知识，因此只能通过本地的替代模型 $F'$ 来生成对抗样本 $X_* = X + \delta X$，但对抗样本 $X_*$ 的攻击效果可以被转移到受害者 $F$ 上. 这种可迁移性可以大致分为两类：
        - **(1) 技术内部可迁移性**：如果模型 $F,F'$ 使用相同的机器学习算法（例如都是神经网络，都是 SVM）
        - **(2) 交叉技术可迁移性**：模型 $F,F'$ 是不同的，例如 $F$ 是神经网络，$F'$ 是 SVM
    
    很多研究表明，这种迁移性攻击可以泛化到**非梯度模型**，例如决策树，即通过本地训练的神经网络代理模型（**梯度模型**），基于它构造的对抗样本对于目标模型为决策树、SVM 或者 KNN 的学习算法同样有攻击效果，**交叉技术可迁移性降低了黑盒攻击者为了迫使目标模型产生错误预测所需掌握的知识**，攻击者只需要专注于在本地训练好代替模型，并仔细选择和构造训练数据集即可
    

#### **9.1.3.3 在联合深度学习中基于 GAN 的攻击**

在一些建模场景中，多个建模者会通过联合学习的方式共同训练一个深度学习模型 $F$，大致设定为：
* 每个参与者**在本地保存自己的训练数据集**，**不会发送给其他参与者**，这些数据包含**重要的敏感信息**
* 所有参与者通过**参数服务器**（**Parameter Server，PS**）共享模型 $M$ 的信息，每个人可以从 PS 下载一份最新的模型参数 $\theta$
* 参与者从 PS 处获得模型，然后在本地执行优化算法（例如 SGD）计算梯度 $g$，然后将梯度上传至 PS
* PS 收集来自哥哥参与者的梯度，然后更新模型的参数

下图展示了 GAN 在联合深度学习中的攻击过程，GAN 可以生成另一个参与者才拥有的样本，从而在联合学习过程中抽取其他参与者的敏感数据和信息，形式化表述攻击过程如下：
- **攻击者 A 和受害者 V 都只能限制访问 PS 中的共享参数**
- 对于受害者 V，他迭代地从 PS 下载参数 $\theta_d$，然后在本地利用自己拥有的样本（例如包含 `class a` 和 `class b` 的样本）训练模型，计算更新模型所需的梯度信息 $\theta_u$，然后将梯度 $\theta_u$ 上传至 PS
- 对于攻击者 A，他迭代地从 PS 下载参数 $\theta_d$，然后将其**拷贝一份到 GAN 的判别器 $D$ 中，那么判别器 $D$ 就拥有判断所有类别数据的能力**，假设攻击者 A 想要从受害者 V 处获得 `class a` 的数据信息，GAN 的生成器 $G$ 就负责尽可能生成能够欺骗过判别器 $D$ 的样本
- 攻击者 A 同样参与合作模型的训练过程，此时除了提供自己本地拥有的样本参与训练外，他**还可以将 GAN 生成的伪造样本也作为训练样本的一部分，并替换其真实标签**（例如生成器 $G$ 的目标是生成 `class a` 的样本，则将生成样本对应的标签从 `class a` 替换为 `class c`），该操作**将有害数据作为噪声注入合作模型，会导致合作模型对 class a 的预测准确度下降**
- 对于**受害者 V，在其观察到更新后的合作模型对 class a 的识别不准确后，就不得不贡献出更多的 `class a` 的真实和敏感信息参与训练**，而攻击者 A 在这个过程中将会获得更丰富的信息，直到 GAN 的生成器 $G$ 能够完全愚弄判别器

<img src="../source/Chap9/基于 GAN 在联合学习中的攻击.jpg" width=1200 style="display: block; margin-left: auto; margin-right: auto;">

#### **9.1.3.4 攻击协同过滤系统**

推荐和协同过滤系统是电商、流媒体、短视频等电商公司的重要模型，这些系统的表现很大程度上决定公司商业化是否成功，因此它们成为了攻击者的重点关注领域. Li 等人之处，当攻击者拥有学习器的完整知识时，可以通过毒害攻击生成恶意样本，极大损害推荐和协同过滤系统

一种最常见的基于因子分解的协同过滤系统原理如下，通过将反馈矩阵 $M$ 做分解得到关于 user 和 item 的特征表示：
$$
\mathop{\min_{U\in \mathbb{R}^{m\times k}, V\in\mathbb{R}^{n\times k}}}
\big\{ \|\mathcal{R}_{\Omega}(M - UV^T)\|_F^2 + 2\lambda_U\| U \|_F^2 + 2\lambda_V \| V\|_F^2 \big\}
$$

其中，$\mathcal{R}_{\Omega}$ 表示可观测数据集，$M\in\mathbb{R}^{m\times n}$ 反馈矩阵 / 评分矩阵，$M_{i,j}$ 表示第 $i$ 个 user 对第 $j$ 个 item 的反馈 / 评分，$k \geq 1$ 是超参数，控制特征表示的隐藏维度，$\lambda_U,\lambda_V > 0$ 是两个正则化参数 

基于协同过滤系统的特点，Li 等人给出了三种类型的攻击，它们的目标效用函数如下：
- **可用性攻击**（**Availability Attack**）：该攻击的目的是最大化协同过滤系统的误差，使系统变得不可靠或不可用. 效用函数定义如下，表示预测矩阵 $\overline{M}$ （**没有做毒害攻击时的预测**）与 $\hat{M}$（**数据被毒害后的预测**）在未观测数据集 $\mathcal{R}_{\Omega_C}$ 上的误差
    $$
    R^{av}(\hat{M},M) = \| \mathcal{R}_{\Omega_C}(\hat{M} - \overline{M})\|_F^2
    $$
- **完整性攻击**（**Integrity Attack**）：攻击者试图操纵 item 子集的受欢迎程度，因此，对于 item 的一个子集 $J_0\subset [n]$ 以及预定义的权重向量 $w:J_0\to \mathbb{R}$，效用函数为：
    $$
    R_{J_0,w}^{in}(\hat{M},M) = \sum_{i=1}^{m}\sum_{j\in J_0} w(j)\hat{M}_{i,j}
    $$
- **混合攻击**（**Hybrid Attack**）：这是前面两种攻击的混合，效用函数表示为两个效用的加权和：
    $$
    R_{J_0,w,\mu}^{hybrid}(\hat{M},M) = \mu_1\cdot R^{av}(\hat{M},M) + \mu_2\cdot R_{J_0,w}^{in}(\hat{M},M)
    $$
    其中，$\mu=(\mu_1,\mu_2)$ 用于平衡两种攻击的效用函数

#### **9.1.3.5 对异常检测算法的攻击**

异常检测算法的目标是确认测试样本 $x$ 是否来自于给定数据集 $X$ 相同的分布，通常当 $x$ 落入到数据集 $X$ 概率密度函数的低密度区域时，异常检测算法会将其视为 outlier 并抛出异常. Kloft 等人使用**有限窗口滑动**（**finite sliding window**），对于在线更新的异常检测系统，训练数据集会不断新增新的样本点，则样本质心将会改变：
$$
c' = c + \frac{1}{n}(x - x_i)
$$

如下图所示，攻击者将尝试迫使异常检测系统接受攻击点 $A$，而样本点 $A$ 落在了正常区域之外，即 $\|A - c\| > r$

<img src="../source/Chap9/攻击异常检测算法.jpg" width=1000 style="display: block; margin-left: auto; margin-right: auto;">

----

### **练习**

1. 阅读对抗攻击和防御综述论文[《Adversarial Attacks and Defences: A Survey》](https://arxiv.org/pdf/1810.00069.pdf)的**第 5 章**，了解对抗学习中**防御方的基本方法**
    * 你也可以在 `../source/Chap9/` 目录下找到该论文